# Telstra Network Disruptions 

The goal of the problem is to predict Telstra network's fault severity at a time at a particular location based on the log data available. Each row in the main dataset (train.csv, test.csv) represents a location and a time point. They are identified by the "id" column, which is the key "id" used in other data files. 

Fault severity has 3 categories: 0,1,2 (0 meaning no fault, 1 meaning only a few, and 2 meaning many). 

Different types of features are extracted from log files and other sources: event_type.csv, log_feature.csv, resource_type.csv, severity_type.csv. 

Note: “severity_type” is a feature extracted from the log files (in severity_type.csv). Often this is a severity type of a warning message coming from the log. "severity_type" is categorical. It does not have an ordering. “fault_severity” is a measurement of actual reported faults from users of the network and is the target variable (in train.csv).

File descriptions
- train.csv - el conjunto de entrenamiento para la severidad de la falla
- test.csv - el conjunto de prueba para la severidad de la falla
- sample_submission.csv – una muestra del formato correcto para la entrada
- event_type.csv: tipo de evento relacionado con el conjunto de datos principal
- log_feature.csv - características extraídas de los archivos de registro
- resource_type.csv: tipo de recurso relacionado con el conjunto de datos principal
- severity_type.csv: tipo de severidad de un mensaje de advertencia que proviene delregistro

# Importamos librerías y datos:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

%matplotlib inline
sns.set()

In [ ]:
train = pd.read_csv('../input/telstra-recruiting-network/train.csv.zip')
train.head()

In [ ]:
test= pd.read_csv('../input/telstra-recruiting-network/test.csv.zip')
test.head()

In [ ]:
sample= pd.read_csv('../input/telstra-recruiting-network/sample_submission.csv.zip')
sample.head()

In [ ]:
sample.tail()

In [ ]:
event= pd.read_csv('../input/telstra-recruiting-network/event_type.csv.zip')
event.head()

In [ ]:
log_feature= pd.read_csv('../input/telstra-recruiting-network/log_feature.csv.zip')
log_feature.head()

In [ ]:
resource_type= pd.read_csv('../input/telstra-recruiting-network/resource_type.csv.zip')
resource_type.head()

In [ ]:
severity_type= pd.read_csv('../input/telstra-recruiting-network/severity_type.csv.zip')
severity_type.head()

In [ ]:
#Para ver los shapes:
print ('train: ', train.shape)
print ('test: ', test.shape)
print ('sample: ', sample.shape)
print ('event: ', event.shape)
print ('log_feature: ', log_feature.shape)
print ('resource_type: ', resource_type.shape)
print ('severity_type: ', severity_type.shape)

Juntando para **train**:

In [ ]:
train1= train.merge(severity_type, how= 'left', left_on='id', right_on='id')
train1.head()

In [ ]:
train2= train1.merge(event, how= 'left', left_on='id', right_on='id')
train2.head()

In [ ]:
train3= train2.merge(resource_type, how= 'left', left_on='id', right_on='id')
train3.head()

In [ ]:
train4= train3.merge(log_feature, how='left', left_on='id', right_on='id')
train4.head()

Podemos ver que hay registros repetidos, por lo que procedemos a eliminamos de acuerdo al `id`.

In [ ]:
train4.drop_duplicates(subset='id', inplace=True)

In [ ]:
train4.reset_index(inplace=True, drop=True)

In [ ]:
train4.head()

In [ ]:
train4.shape

Junatmos para **test**

In [ ]:
test.head()

In [ ]:
test1= test.merge(severity_type, how= 'left', left_on='id', right_on='id')
test1.head()

In [ ]:
test2= test1.merge(event, how= 'left', left_on='id', right_on='id')
test2.head()

In [ ]:
test3= test2.merge(resource_type, how= 'left', left_on='id', right_on='id')
test3.head()

In [ ]:
test4= test3.merge(log_feature, how='left', left_on='id', right_on='id')
test4.head()

Elimimos duplicados de test:

In [ ]:
test4.drop_duplicates(subset='id', inplace=True)

In [ ]:
test4.reset_index(inplace=True, drop=True)

In [ ]:
test4.head()

In [ ]:
test.shape

Para ver la info de train y de test:

In [ ]:
train4.info()

In [ ]:
test4.info()

In [ ]:
train4_cols= train4.columns.tolist()
train4_cols

### Valores faltantes

In [ ]:
#Vamos a ver si hay valores faltantes
train4.isnull().sum()

In [ ]:
#con un heatmap de seaborn
sns.heatmap(train4.isnull(), yticklabels=False, cbar=False, cmap="Blues")

In [ ]:
#Vamos a ver si hay valores faltantes
test4.isnull().sum()

In [ ]:
#con un heatmap de seaborn
sns.heatmap(test4.isnull(), yticklabels=False, cbar=False, cmap="Blues")

## Análisis Exploratorio de Datos (EDA)

In [ ]:
#Para ver los valores únicos por columna:
print ('Uniques para location: ', train4.location.unique())
print ('Uniques para fault_severity: ', train4.fault_severity.unique())
print ('Uniques para severity_type: ', train4.severity_type.unique())
print ('Uniques para event_type: ', train4.event_type.unique())
print ('Uniques para resource_type: ', train4.resource_type.unique())
print ('Uniques para log_feature: ', train4.log_feature.unique())
print ('Uniques para volume: ', train4.volume.unique())

## Un vistazo general (A general view).

Para ver la cantidad de casos según **fault_severity**:

In [ ]:
#Para visualizar los datos según l2
t4_fs = train4.groupby('fault_severity', sort=False, as_index=False).size().sort_values(by="size",ascending=False)
t4_fs

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data = train4, x='fault_severity', lw=1, edgecolor="black")
plt.title ('Cantidad de casos por Gravedad de Falla (Fault Severity)')
plt.xlabel('Gravedad de Falla')
plt.ylabel('Cantidad')
plt.show()

Podemos ver que hay mayor cantidad de no fallas que de fallas 1 o 2.

Para ver la cantidad de casos según **severity_type**:

In [ ]:
t4_st = train4.groupby('severity_type', sort=False, as_index=False).size().sort_values(by="size",ascending=False)
t4_st

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data = train4, x='severity_type', lw=1, edgecolor="black")
plt.title ('Cantidad de casos por Tipo de Gravedad (Severity Type)')
plt.xlabel('Tipo de Gravedad')
plt.ylabel('Cantidad')
plt.show()

C on esta información podemos ver que las fallas que más se presentan son las de tipo 1 y 2. Las que gravedades tipo 4, 5 y 3 se presentan muy poco (en comparación con las tipo 1 y 2).

Para ver la cantidad de casos según **resource_type**:

In [ ]:
t4_rt = train4.groupby('resource_type', sort=False, as_index=False).size().sort_values(by="size",ascending=False)
t4_rt

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data = t4_rt, x='resource_type', y= 'size',lw=1, edgecolor="black")
plt.title ('Cantidad de casos por Tipo de Recurso (Resource Type)')
plt.xlabel('Tipo de Recurso')
plt.ylabel('Cantidad')
plt.show()

Los tipos de recursos mas utilizados son: 2 y 8. Mientras que el resto de recursos tienen relativamente pocos casos.

Para ver la cantidad de casos según **location**:

In [ ]:
t4_loc = train4.groupby('location', sort=False, as_index=False).size().sort_values(by="size",ascending=False)
t4_loc

Las 5 locaciones que tienen más casos son: 821, 1107, 734, 126 y 1008.

## Fallas tipo 1 (fault_severity = 1)

Vamos a analizar las fallas tipo 1 para ver qué podemos encontrar

In [ ]:
fallas_tipo_1= train4[train4.fault_severity == 1]

In [ ]:
fallas_tipo_1.head()

In [ ]:
fallas_tipo_1.shape

In [ ]:
ft1_st = fallas_tipo_1.groupby('severity_type', sort=False, as_index=False).size().sort_values(by="size",ascending=False)
ft1_st

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data = ft1_st, x='severity_type', y='size', lw=1, edgecolor="black")
plt.title ('Cantidad de casos por Tipo de Gravedad (Severity Type)')
plt.xlabel('Tipo de Gravedad')
plt.ylabel('Cantidad')
plt.show()

Los mensajes que más se muestran son los tipo 1

In [ ]:
ft1_rt = fallas_tipo_1.groupby('resource_type', sort=False, as_index=False).size().sort_values(by="size",ascending=False)
ft1_rt

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data = ft1_rt, x='resource_type', y='size', lw=1, edgecolor="black")
plt.title ('Cantidad de casos por Tipo de Recurso (Resource Type)')
plt.xlabel('Tipo de Recurso')
plt.ylabel('Cantidad')
plt.show()

Las fallas con severidad 1, tienen mayor cantidad de recursos tipo 8 y 2.

In [ ]:
ft1_loc = fallas_tipo_1.groupby('location', sort=False, as_index=False).size().sort_values(by="size",ascending=False)
ft1_loc

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data = ft1_loc[1:15], x='location', y='size', lw=1, edgecolor="black")
plt.title ('15 Locaciones con más casos de fallas tipo 1')
plt.xlabel('Locaciones')
plt.xticks(rotation=90)
plt.ylabel('Cantidad')
plt.show()

In [ ]:
ft1_et = fallas_tipo_1.groupby('event_type', sort=False, as_index=False).size().sort_values(by="size",ascending=False)
ft1_et

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data = ft1_et, x='event_type', y='size', lw=1, edgecolor="black")
plt.title ('Cantidad de casos por Tipo de Evento (Event Type)')
plt.xlabel('Tipo de Evento')
plt.xticks(rotation=90)
plt.ylabel('Cantidad')
plt.show()

## Fallas tipo 2 (fault_severity = 2)

Vamos a analizar las fallas tipo 1 para ver qué podemos encontrar

In [ ]:
fallas_tipo_2= train4[train4.fault_severity == 2]

In [ ]:
fallas_tipo_2.head()

In [ ]:
fallas_tipo_2.shape

In [ ]:
ft2_st = fallas_tipo_2.groupby('severity_type', sort=False, as_index=False).size().sort_values(by="size",ascending=False)
ft2_st

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data = ft2_st, x='severity_type', y='size', lw=1, edgecolor="black")
plt.title ('Cantidad de casos por Tipo de Gravedad (Severity Type)')
plt.xlabel('Tipo de Gravedad')
plt.ylabel('Cantidad')
plt.show()

In [ ]:
ft2_rt = fallas_tipo_2.groupby('resource_type', sort=False, as_index=False).size().sort_values(by="size",ascending=False)
ft2_rt

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data = ft2_rt, x='resource_type', y='size', lw=1, edgecolor="black")
plt.title ('Cantidad de casos por Tipo de Recurso (Resource Type)')
plt.xlabel('Tipo de Recurso')
plt.ylabel('Cantidad')
plt.show()

In [ ]:
ft2_loc = fallas_tipo_2.groupby('location', sort=False, as_index=False).size().sort_values(by="size",ascending=False)
ft2_loc

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data = ft2_loc[1:15], x='location', y='size', lw=1, edgecolor="black")
plt.title ('15 Locaciones con más casos de fallas tipo 1')
plt.xlabel('Locaciones')
plt.xticks(rotation=90)
plt.ylabel('Cantidad')
plt.show()

In [ ]:
ft2_et = fallas_tipo_2.groupby('event_type', sort=False, as_index=False).size().sort_values(by="size",ascending=False)
ft2_et

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data = ft2_et, x='event_type', y='size', lw=1, edgecolor="black")
plt.title ('Cantidad de casos por Tipo de Evento (Event Type)')
plt.xlabel('Tipo de Evento')
plt.xticks(rotation=90)
plt.ylabel('Cantidad')
plt.show()

Podemos ver que los casos que registran fallas tipo 1 y los casos que registran fallas tipo 2 son diferentes en los tipos de eventos, locaciones, tipo de severidad.

Mientras que para el tipo de recurso son iguales en ambos casos (8 y 2)

## Preprocesamiento de datos (Data Preprocesing).

In [ ]:
train4.head()

Pasamos los datos a valores:

In [ ]:
train4['location']= train4.location.str.extract('(\d+)', expand=False)
train4['location']= train4.location.astype(int)

train4['severity_type']= train4.severity_type.str.extract('(\d+)', expand=False)
train4['severity_type']= train4.severity_type.astype(int)

train4['event_type']= train4.event_type.str.extract('(\d+)', expand=False)
train4['event_type']= train4.event_type.astype(int)

train4['resource_type']= train4.resource_type.str.extract('(\d+)', expand=False)
train4['resource_type']= train4.resource_type.astype(int)

train4['log_feature']= train4.log_feature.str.extract('(\d+)', expand=False)
train4['log_feature']= train4.log_feature.astype(int)

train4.head()

In [ ]:
X= train4.drop(['id', 'fault_severity'], axis=1)

In [ ]:
X.head()

In [ ]:
X.describe()

Podemos ver que hay valores muy grandes para las columnas de `location`, `log_feature` y `volume`, por lo que se aplica MinMaxScaler para que se de mayor peso a estas columnas sobre las otras.

In [ ]:
y= train4.fault_severity

In [ ]:
print ('X shape: ', X.shape)
print ('y shape: ', y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.75)

In [ ]:
print ('X_train shape: ', X_train.shape)
print ('y_train shape: ', y_train.shape)
print ('X_test shape: ', X_test.shape)
print ('y_test shape: ', y_test.shape)

Preprocesamos para predecir con el dataset de **test**:

In [ ]:
test4.head()

In [ ]:
test4['location']= test4.location.str.extract('(\d+)', expand=False)
test4['location']= test4.location.astype(int)

test4['severity_type']= test4.severity_type.str.extract('(\d+)', expand=False)
test4['severity_type']= test4.severity_type.astype(int)

test4['event_type']= test4.event_type.str.extract('(\d+)', expand=False)
test4['event_type']= test4.event_type.astype(int)

test4['resource_type']= test4.resource_type.str.extract('(\d+)', expand=False)
test4['resource_type']= test4.resource_type.astype(int)

test4['log_feature']= test4.log_feature.str.extract('(\d+)', expand=False)
test4['log_feature']= test4.log_feature.astype(int)

test4.head()

In [ ]:
X_predic = test4.drop('id', axis=1)

In [ ]:
X_predic.head()

In [ ]:
X_predic.shape

### Elección de Métricas (KPIs de evaluación de modelos)

Para un `modelo de clasificación` vamos a utilizar la `matriz de confusión` que nos muestra la forma en que clasificó el modelo que estamos entrenando. A partir de la matriz vamos a revisar 4 indicadores y prestar especial atención a 2 de ellos:

**Accuracy (Exactitud):** Nos dice qué tan bien predijo el modelo. Se representa como  la proporción de resultados verdaderos (tanto verdaderos positivos (TP) como verdaderos negativos (TN)) dividido entre el número total de casos examinados.

En forma práctica,  el accuracy es  la cantidad de predicciones que fueron correctas.

$$Accuracy = \frac{TP+TN}{TP+TN+FP+FN}$$

**Precision (Precisión):**  Se refiere a la dispersión del conjunto de valores obtenidos a partir de mediciones repetidas de una magnitud. Cuanto menor es la dispersión mayor la precisión. Se representa por la proporción de verdaderos positivos dividido entre todos los resultados positivos (tanto verdaderos positivos, como falsos positivos).En forma práctica es  el porcentaje de casos positivos detectados.

$$Precision = \frac{VP}{VP+FP}$$

**Recall (Sensibilidad):** Es un valor que nos indican la capacidad de nuestro modelo para discriminar los casos positivos, de los negativos. Es la proporción de casos positivos que fueron correctamente identificadas por el algoritmo.

$$Recall = \frac{VP}{VP+FN}$$  

O lo que sería igual :  "Verdaderos positivos / Total de enfermos" (en el área de la salud se dice que la sensibilidad es la capacidad de de poder detectar correctamente la enfermedad entre los enfermos).

**F1 Score:** Es otra métrica muy empleada porque resume la precisión y recall en una sola métrica. Por ello es de gran utilidad cuando la distribución de las clases es desigual (datasets desbalanceados).

$$F1 \ score \ =  \frac{2 * (Recall * Precision)}{(Recall + Precision)}$$

Creamos una función para evaluar los resultados:

In [ ]:
def resultados (y_test, y_pred):
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(4, 4))
    sns.heatmap(conf_matrix, cbar= False,  square= True, annot=True, fmt= '.0f', annot_kws={'size': 15},
           cmap= 'coolwarm', linewidths=.5);
    plt.title('Matriz de Confusión')
    plt.ylabel('Predicción')
    plt.xlabel('Clase Real')
    plt.show()
    print (classification_report(y_test, y_pred))

## Modelo Benchmark

### Decision Tree

In [ ]:
tree= DecisionTreeClassifier(random_state=42)

In [ ]:
tree.fit(X_train, y_train)

In [ ]:
y_pred_tree= tree.predict(X_test)

In [ ]:
resultados (y_test, y_pred_tree)

### KNN

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
y_pred_knn= knn.predict(X_test)

In [ ]:
resultados (y_test, y_pred_knn)

### Random Forest

In [ ]:
forest = RandomForestClassifier(random_state=42)

In [ ]:
forest.fit(X_train, y_train)

In [ ]:
y_pred_forest= forest.predict(X_test)

In [ ]:
resultados (y_test, y_pred_forest)

## Mejorando el mejor modelo (Random Forest)

El mejor modelo fue el de Random Forest, vamos a mejorarlo utilizando GridSearchCV.

In [ ]:
#Hacemos la grilla para Grid Search:
param_grid = {'n_estimators': [200, 300, 400, 500],   
              'min_samples_split': [2,3,4,5],    
              'min_samples_leaf':[1,3,5]}   

In [ ]:
model = GridSearchCV(forest, param_grid=param_grid, cv=5)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
print("Mejores parametros: "+str(model.best_params_))
print("Mejor Score: "+str(model.best_score_)+'\n')

Creamos un modelo nuevo con los mejores parámetros.

In [ ]:
forest2= RandomForestClassifier(min_samples_leaf=5, min_samples_split=2, n_estimators=400, random_state=42)

In [ ]:
forest2.fit(X_train, y_train)

In [ ]:
y_pred_forest2= forest2.predict(X_test)

In [ ]:
resultados (y_test, y_pred_forest2)

Para ver los estimadores:

In [ ]:
forest2.feature_importances_

In [ ]:
cols_model= ['location','severity_type','event_type','resource_type','log_feature','volume']
cols_model

In [ ]:
feature_importance= pd.DataFrame(list(zip(cols_model, forest2.feature_importances_.transpose())), columns = ['Col','Importance']).sort_values(by="Importance",ascending=False)
feature_importance

## Predicción con el dataset de test

In [ ]:
X_predic.head()

In [ ]:
prediccion= forest2.predict_proba(X_predic)
prediccion

In [ ]:
#Lo pasamos a dataframe
prediccion_data= pd.DataFrame(prediccion,columns=['predict_0', 'predict_1', 'predict_2'])
prediccion_data

In [ ]:
#Lo juntamos con el id
submission= pd.concat([test[['id']],prediccion_data],axis=1)
submission

In [ ]:
#Lo guardamos en el archivo:
#submission.to_csv('submission.csv',index=False,header=True)